In [1]:
import numpy as np
import os
import glog as log
import copy

from __future__ import division

import open3d as o3d
from open3d import JVisualizer
import pandas as pd

from evaluation.tools.mesh import Mesh
from evaluation.tools.mesh_evaluator import MeshEvaluator

# Rotation matrices:
# East North Up (ENU) frame to Unity's world frame of reference
enu_R_unity = np.array([[1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]])
unity_R_enu = np.transpose(enu_R_unity)

# Right Handed frame to Unity's Left Handed frame of reference
righthand_R_lefthand = np.array([[1, 0, 0],
                                 [0, -1, 0],
                                 [0, 0, 1]])
lefthand_R_righthand = np.transpose(righthand_R_lefthand)

In [2]:
def read_pcl(pcl_path):
    return o3d.io.read_point_cloud(pcl_path)

def read_mesh(mesh_path):
    return Mesh(mesh_path)

In [3]:
def sample_mesh(mesh):
    NUMBER_OF_SAMPLES=100000
    return o3d.geometry.sample_points_uniformly(mesh.mesh_o3d, NUMBER_OF_SAMPLES)

In [4]:
def visualize_pcls(est_pcl, gt_pcl):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.get_render_option().mesh_show_back_face = True
    vis.add_geometry(est_pcl)
    vis.add_geometry(gt_pcl)
    vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
    vis.run()
    vis.destroy_window()

In [5]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [7]:
# Visualize initial registration problem
def visualize_registration(est_pcl, gt_pcl, initial_transformation):
    draw_registration_result(est_pcl, gt_pcl, initial_transformation)

In [8]:
# Evaluate current fit between pointclouds
def evaluate_registration(est_pcl, gt_pcl, icp_threshold, initial_transformation):
    evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, icp_threshold, initial_transformation)
    #print("Initial registration")
    print(evaluation)

In [9]:
def p2p_icp(est_pcl, gt_pcl, icp_threshold, initial_transformation):
    #print("Apply point-to-point ICP")
    return o3d.registration.registration_icp(
        est_pcl, gt_pcl, icp_threshold, initial_transformation,
        o3d.registration.TransformationEstimationPointToPoint(),
        o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))

In [14]:
# ICP params
ICP_THRESHOLD = 0.5
import math
yaw = 0
tx = 0
ty = 0
tz = 0
if False:
    yaw = 44
    tx = 18.5
    ty = 25
    tz = 2.5
yaw_rad = math.radians(yaw)
trans_init = np.asarray([[math.cos(yaw_rad), -math.sin(yaw_rad), 0.0, tx],
                         [math.sin(yaw_rad), math.cos(yaw_rad), 0.0, ty],
                         [0.0, 0.0, 1.0, tz],
                         [0.0, 0.0, 0.0, 1.0]])

In [ ]:
# FILL PATHS BELOW
EUROC_DATASET_PATH = "/home/tonirv/datasets/euroc/EuRoC/"
euroc_dataset_names = ["V1_01_easy", "V1_02_medium", "V1_03_difficult", "V2_01_easy", "V2_02_medium", "V2_03_difficult"]

results_appendix = ["_spark_vio_registered.ply", "_time_horizon_mesh.ply"]

MESH_TYPE = 0 # 0: global 1: time-horizon

calculate_precision = True
calculate_recall = True

precision = {euroc_dataset: 0 for euroc_dataset in euroc_dataset_names}
recall = {euroc_dataset: 0 for euroc_dataset in euroc_dataset_names}

for euroc_dataset in euroc_dataset_names:
    gt_pcl_path = os.path.join(EUROC_DATASET_PATH, euroc_dataset, "mav0/pointcloud0/data.ply")
    filename = euroc_dataset + results_appendix[MESH_TYPE]
    est_pcl_path = os.path.join("/home/tonirv/Documents/Research/Papers/2019w-IROS-SparkVIO/results/CloudCompare", filename)
    
    print("Loading Ground-truth and Estimated meshes from: ") 
    print("- GT PCL: %s"% gt_pcl_path)
    gt_pcl_original = read_pcl(gt_pcl_path)
    print("- EST Mesh: %s"% est_pcl_path)
    if MESH_TYPE == 0:
        est_pcl_original = read_pcl(est_pcl_path)
    elif MESH_TYPE == 1:
        est_mesh = read_mesh(est_pcl_path)
        #o3d.visualization.draw_geometries([est_mesh.mesh_o3d])
        est_pcl_original = sample_mesh(est_mesh)
        #o3d.visualization.draw_geometries([est_pcl_original])
    else:
        raise
        
    # Visualize pointclouds
    #o3d.visualization.draw_geometries([gt_pcl, est_pcl])
    
    print("RESULTS FOR: %s" % euroc_dataset)
    if calculate_precision:
        print("Precision: ")
        reg_p2p = p2p_icp(est_pcl_original, gt_pcl_original, ICP_THRESHOLD, trans_init)
        precision[euroc_dataset] = {"Fitness": reg_p2p.fitness, "Inlier RMSE": reg_p2p.inlier_rmse}
        print(reg_p2p)
    if calculate_recall:
        print("Recall: ")
        reg_p2p = p2p_icp(gt_pcl_original, est_pcl_original, ICP_THRESHOLD, trans_init)
        recall[euroc_dataset] = {"Fitness": reg_p2p.fitness, "Inlier RMSE": reg_p2p.inlier_rmse}
        print(reg_p2p)

In [ ]:
print precision

In [ ]:
print recall